In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Đổi thư mục này. Mapping với drive
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/dataset'):
    print("dir name",dirname)
    for filename in filenames:
        print(os.path.join(dirname, filename))

dir name /content/drive/MyDrive/Colab Notebooks/dataset
dir name /content/drive/MyDrive/Colab Notebooks/dataset/train
/content/drive/MyDrive/Colab Notebooks/dataset/train/Recon-Ping_Sweep_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-UDP4_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/Recon-VulScan_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-SYN3_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-UDP3_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-TCP3_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-SYN1_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-TCP2_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-SYN4_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/dataset/train/TCP_IP-DoS-TCP1_train.pcap.csv
/content/drive/MyDrive/Colab Notebooks/data

In [2]:
del os

In [3]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [4]:
import os

def merge_files_in_folder(folder_path):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        # Extract label from filename and remove numbers at the end
        label = filename.split("_train.pcap.csv")[0].split("_test.pcap.csv")[0].split("1234567890")[0]
                        # Remove numeric part if it exists
        label = ''.join(filter(str.isalpha, label))
        # Read the file into a DataFrame
        filepath = os.path.join(folder_path, filename)
        df = pd.read_csv(filepath)

        # Add a 'label' column with the current label
        df['label'] = label

        # Append the DataFrame to the list
        dfs.append(df)

    # Concatenate all DataFrames in the list
    merged_data = pd.concat(dfs, ignore_index=True)
    del dfs

    # Display value counts of labels
    print("Value counts of labels in train data:")
    print(merged_data['label'].value_counts())
    return merged_data


# Merge files in the train folder
train_folder_path = "/content/drive/MyDrive/Colab Notebooks/dataset/train"
test_folder_path = "/content/drive/MyDrive/Colab Notebooks/dataset/test"

train_df = merge_files_in_folder(train_folder_path)
test_df = merge_files_in_folder(test_folder_path)

Value counts of labels in train data:
label
TCPIPDDoSUDP            1635956
TCPIPDDoSICMP           1537476
TCPIPDDoSTCP             804465
TCPIPDDoSSYN             801962
TCPIPDoSSYN              441903
TCPIPDoSICMP             416292
TCPIPDoSUDP              289447
TCPIPDoSTCP              286896
Benign                   192732
MQTTDDoSConnectFlood     173036
ReconPortScan             83981
MQTTDoSPublishFlood       44376
MQTTDDoSPublishFlood      27623
ReconOSScan               16832
ARPSpoofing               16047
MQTTDoSConnectFlood       12773
MQTTMalformedData          5130
ReconVulScan               2173
ReconPingSweep              740
Name: count, dtype: int64
Value counts of labels in train data:
label
TCPIPDDoSUDP            362070
TCPIPDDoSICMP           349699
TCPIPDDoSTCP            182598
TCPIPDDoSSYN            172397
TCPIPDoSUDP             137553
TCPIPDoSSYN              98595
TCPIPDoSICMP             98432
TCPIPDoSTCP              82096
MQTTDDoSConnectFlood     41916

In [5]:
df_head = train_df.head()
df_info= train_df.info()
df_shape=train_df.shape

print(df_head)
print(df_info)
print(df_shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6789840 entries, 0 to 6789839
Data columns (total 46 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Header_Length    float64
 1   Protocol Type    float64
 2   Duration         float64
 3   Rate             float64
 4   Srate            float64
 5   Drate            float64
 6   fin_flag_number  float64
 7   syn_flag_number  float64
 8   rst_flag_number  float64
 9   psh_flag_number  float64
 10  ack_flag_number  float64
 11  ece_flag_number  float64
 12  cwr_flag_number  float64
 13  ack_count        float64
 14  syn_count        float64
 15  fin_count        float64
 16  rst_count        float64
 17  HTTP             float64
 18  HTTPS            float64
 19  DNS              float64
 20  Telnet           float64
 21  SMTP             float64
 22  SSH              float64
 23  IRC              float64
 24  TCP              float64
 25  UDP              float64
 26  DHCP             float64
 27  ARP         

In [6]:
merged_df = pd.concat([train_df, test_df], ignore_index=True)
merged_df.head()

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IGMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,199.1,7.1,80.8,44193.400088,44193.400088,0.0,0.0,0.0,0.0,0.5,0.9,0.0,0.0,0.0,0.0,0.0,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.1,0.0,0.0,0.0,0.0,1.0,1.0,1232.6,79.2,847.5,254.414286,290.225086,165.3,1.694105e+08,5.5,22.062066,410.440253,95884.745554,0.9,38.5,ReconPingSweep
1,163.6,7.1,70.4,118249.943542,118249.943542,0.0,0.0,0.0,0.0,0.5,0.9,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.1,0.0,0.0,0.0,0.0,1.0,1.0,2176.0,66.0,928.0,141.805497,211.947485,108.8,1.694105e+08,13.5,16.848683,300.383675,45391.533968,1.0,244.6,ReconPingSweep
2,289.2,6.0,64.0,3.502257,3.502257,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,440.9,66.0,101.1,78.209802,14.646538,80.5,5.838871e-05,5.5,12.499571,20.713332,251.804006,0.9,38.5,ReconPingSweep
3,278.4,7.1,80.8,11188.114047,11188.114047,0.0,0.0,0.0,0.0,0.3,0.9,0.0,0.0,0.0,0.0,0.0,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.1,0.0,0.0,0.0,0.0,1.0,1.0,1334.4,66.0,203.7,86.075004,34.778234,89.3,1.694105e+08,13.5,13.126486,49.266605,1254.068664,1.0,244.6,ReconPingSweep
4,375.0,6.0,64.0,12522.810687,12522.810687,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,0.0,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,447.8,70.5,111.0,85.157381,15.187175,78.3,7.510185e-05,5.5,13.032968,21.477909,261.779852,0.9,38.5,ReconPingSweep


In [7]:
merged_df.shape
merged_df['label'].value_counts()

,count
label,
TCPIPDDoSUDP,1998026
TCPIPDDoSICMP,1887175
TCPIPDDoSTCP,987063
TCPIPDDoSSYN,974359
TCPIPDoSSYN,540498
TCPIPDoSICMP,514724
TCPIPDoSUDP,427000
TCPIPDoSTCP,368992
Benign,230339


In [8]:
# Assuming train_df is your DataFrame
train_df = train_df.sample(frac=1).reset_index(drop=True)

# Assuming test_df is your DataFrame
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [9]:
# Check for missing values
missing_values = train_df.isnull().sum()

# Check for duplicate rows
duplicates = train_df.duplicated().sum()

print("missing values:",missing_values)
print("duplicate values:",duplicates)

del missing_values
del duplicates

missing values: Header_Length      0
Protocol Type      0
Duration           0
Rate               0
Srate              0
Drate              0
fin_flag_number    0
syn_flag_number    0
rst_flag_number    0
psh_flag_number    0
ack_flag_number    0
ece_flag_number    0
cwr_flag_number    0
ack_count          0
syn_count          0
fin_count          0
rst_count          0
HTTP               0
HTTPS              0
DNS                0
Telnet             0
SMTP               0
SSH                0
IRC                0
TCP                0
UDP                0
DHCP               0
ARP                0
ICMP               0
IGMP               0
IPv                0
LLC                0
Tot sum            0
Min                0
Max                0
AVG                0
Std                0
Tot size           0
IAT                0
Number             0
Magnitue           0
Radius             0
Covariance         0
Variance           0
Weight             0
label              0
dtype: int64
dupli

In [10]:
missing_values = test_df.isnull().sum()

duplicates = test_df.duplicated().sum()

print("missing values:",missing_values)
print("duplicate values:",duplicates)

del missing_values
del duplicates

missing values: Header_Length      0
Protocol Type      0
Duration           0
Rate               0
Srate              0
Drate              0
fin_flag_number    0
syn_flag_number    0
rst_flag_number    0
psh_flag_number    0
ack_flag_number    0
ece_flag_number    0
cwr_flag_number    0
ack_count          0
syn_count          0
fin_count          0
rst_count          0
HTTP               0
HTTPS              0
DNS                0
Telnet             0
SMTP               0
SSH                0
IRC                0
TCP                0
UDP                0
DHCP               0
ARP                0
ICMP               0
IGMP               0
IPv                0
LLC                0
Tot sum            0
Min                0
Max                0
AVG                0
Std                0
Tot size           0
IAT                0
Number             0
Magnitue           0
Radius             0
Covariance         0
Variance           0
Weight             0
label              0
dtype: int64
dupli

In [11]:
# remove duplicates rồi đi train
train_df = train_df.drop_duplicates()
test_df = test_df.drop_duplicates()

In [ ]:
missing_values = train_df.isnull().sum()
duplicates = train_df.duplicated().sum()

print("missing values:",missing_values)
print("duplicate values:",duplicates)

missing_values = test_df.isnull().sum()
duplicates = test_df.duplicated().sum()

print("missing values:",missing_values)
print("duplicate values:",duplicates)

del missing_values
del duplicates

In [ ]:
train_attack_label_counts = train_df['label'].value_counts()
print(train_attack_label_counts)

In [ ]:
test_attack_label_counts = test_df['label'].value_counts()
print(test_attack_label_counts)

In [ ]:
# Set the style for seaborn plots
import seaborn as sns
sns.set_style('whitegrid')

# Increase default font size for readability
sns.set_context('talk')
# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Bar chart for the distribution of attack labels
plt.figure(figsize=(14, 10))
train_attack_label_counts.plot(kind='bar')
plt.title('Distribution of Attack Labels (train dataset)')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.xticks(rotation=90)  # Rotate the x labels to show them clearly
plt.tight_layout()  # Adjust the plot to ensure everything fits without overlapping

# Show the plot
plt.show()

In [ ]:
sns.set_style('whitegrid')

# Increase default font size for readability
sns.set_context('talk')
# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Bar chart for the distribution of attack labels
plt.figure(figsize=(14, 10))
test_attack_label_counts.plot(kind='bar')
plt.title('Distribution of Attack Labels (test dataset)')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.xticks(rotation=90)  # Rotate the x labels to show them clearly
plt.tight_layout()  # Adjust the plot to ensure everything fits without overlapping

# Show the plot
plt.show()

In [ ]:
# Mapping labels to new categories for 6 classes
label_map_6_classes = {
    "DDos": ["TCPIPDDoSUDP", "TCPIPDDoSICMP", "TCPIPDDoSTCP", "TCPIPDDoSSYN"],
    "Dos": ["TCPIPDoSUDP", "TCPIPDoSSYN", "TCPIPDoSICMP", "TCPIPDoSTCP"],
    "MQTT": ["MQTTDDoSConnectFlood", "MQTTDoSPublishFlood", "MQTTDDoSPublishFlood", "MQTTDoSConnectFlood","MQTTMalformedData"],
    "SPOOFING": ["ARPSpoofing"],
    "RECON": ["ReconPortScan", "ReconOSScan", "ReconVulScan", "ReconPingSweep"],
    "Benign": ["Benign"]
}

# Mapping labels to new categories for 2 classes
label_map_2_classes = {
    "Benign": ["Benign"],
    "Attack": ["TCPIPDDoSUDP", "TCPIPDDoSICMP", "TCPIPDDoSTCP", "TCPIPDDoSSYN", "TCPIPDoSUDP", "TCPIPDoSSYN",
               "TCPIPDoSICMP", "TCPIPDoSTCP", "MQTTDDoSConnectFlood", "ReconPortScan", "MQTTDoSPublishFlood",
               "MQTTDDoSPublishFlood", "ReconOSScan", "ARPSpoofing", "MQTTDoSConnectFlood", "MQTTMalformedData",
               "ReconVulScan", "ReconPingSweep"]
}

# Function to map labels to new categories
def map_labels_to_categories(labels, label_map):
    categories = []
    for label in labels:
        for category, labels_list in label_map.items():
            if label in labels_list:
                categories.append(category)
                break
    return categories

# Map labels to categories for 6 classes
labels_6_class = map_labels_to_categories(train_df['label'], label_map_6_classes)
labels_6_class_test = map_labels_to_categories(test_df['label'], label_map_6_classes)

# Map labels to categories for 2 classes
labels_2_class = map_labels_to_categories(train_df['label'], label_map_2_classes)
labels_2_class_test = map_labels_to_categories(test_df['label'], label_map_2_classes)

In [ ]:
labels_6_class = pd.Series(labels_6_class)
labels_2_class = pd.Series(labels_2_class)
labels_6_class_test = pd.Series(labels_6_class_test)
labels_2_class_test = pd.Series(labels_2_class_test)
print(labels_2_class.shape)
print(labels_6_class.shape)
print(train_df.shape)
print(labels_2_class_test.shape)
print(labels_6_class_test.shape)

In [ ]:
label_counts_6_classes = labels_6_class.value_counts()
print("6 Classes Value Count\n",label_counts_6_classes)
label_counts_2_classes = labels_2_class.value_counts()
print("2 Classes Value Count\n",label_counts_2_classes)

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

# Increase default font size for readability
sns.set_context('talk')
# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Bar chart for the distribution of attack labels
plt.figure(figsize=(10, 6))
label_counts_6_classes.plot(kind='bar')
plt.title('Distribution of Attack Labels (6 Classes)')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.xticks(rotation=90)  # Rotate the x labels to show them clearly
plt.tight_layout()  # Adjust the plot to ensure everything fits without overlapping

# Show the plot
plt.show()

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

# Increase default font size for readability
sns.set_context('talk')
# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Bar chart for the distribution of attack labels
plt.figure(figsize=(10, 6))
label_counts_2_classes.plot(kind='bar')
plt.title('Distribution of Attack Labels (2 Classes)')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.xticks(rotation=90)  # Rotate the x labels to show them clearly
plt.tight_layout()  # Adjust the plot to ensure everything fits without overlapping

# Show the plot
plt.show()

In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
label_encoder1 = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
train_df['label']= label_encoder.fit_transform(train_df['label'])
test_df['label']= label_encoder1.fit_transform(test_df['label'])
labels_6_class= label_encoder.fit_transform(labels_6_class)
labels_2_class= label_encoder.fit_transform(labels_2_class)
labels_6_class_test= label_encoder.fit_transform(labels_6_class_test)
labels_2_class_test= label_encoder.fit_transform(labels_2_class_test)

In [ ]:
X = train_df.drop('label', axis=1)
y = train_df['label']

In [ ]:
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

In [ ]:
del train_df
del test_df

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# Number of classes
num_classes = 19
print(y_test.shape)
print(labels_6_class_test.shape)

# Convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_test_6_class = tf.keras.utils.to_categorical(labels_6_class_test, 6)
y_test_2_class = tf.keras.utils.to_categorical(labels_2_class_test, 2)
y_train_6_class = tf.keras.utils.to_categorical(labels_6_class, 6)
y_train_2_class = tf.keras.utils.to_categorical(labels_2_class, 2)
print(X_test_scaled.shape)
print(y_test.shape)
print(y_test_6_class.shape)

In [ ]:
print(y_train.shape)

In [ ]:
del X
del y

In [ ]:
# Define callbacks
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)

# Training loop
epochs = 10

In [ ]:
num_classes = 19 # So loai tan cong
model = Sequential([
    Dense(64, activation='relu', input_shape=(45,)),
    Dropout(0.2),  # Add dropout regularization
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  # Using softmax activation for multiclass classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model: num_classes, epochs..
history = model.fit(X_train_scaled, y_train, epochs=2, batch_size=32, validation_split=0.2,callbacks=[checkpoint, early_stopping, tensorboard])


In [ ]:
# Accuracy and loss graph
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy / Loss')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Confusion Matrix and Classification Report
y_pred = np.argmax(model.predict(X_test_scaled), axis=-1)
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print("Confusion Matrix:")
print(cm)
print("Classification Report:")
print(classification_report(np.argmax(y_test, axis=1), y_pred))

# Train